# Burrow Fear Conditioning Analysis Pipeline

This is a jupyter notebook to conduct the 'standard' analysis of burrow fear conditioning data.

## Experiment Hierarchy

Here we construct and organizational framework for the various behavioral stages and meta-data.

### Import Modules

In [1]:
from AnalysisModules.ExperimentHierarchy import ExperimentData

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np



### Identify Directory, Study, & Mouse Information

In [2]:
__directory="H:\\DEM_Excitatory_Study\\DEM2"
__mouse="M4584"
__study="DEM"
__study_mouse="DEM2"

### Construct Experiment Hierarchy

In [3]:
Data = ExperimentData(Directory=__directory, Mouse=__mouse, Study=__study, StudyMouse=__study_mouse)

## Conduct Analysis on Encoding

### Import Modules

In [4]:
from AnalysisModules.BurrowFearConditioning import Encoding

In [5]:
Data.Encoding = Encoding(Data.passMeta())

### Identify Dataset and Parameters

In [6]:
__data_folder = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz"
__index_file = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz\\NeuronalIndex.csv"
__features_file = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz\\Features.csv"

### Import Modules

In [7]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule

# CASCADE: Spike Inference
from AnalysisModules.CascadeAnalysis import CascadeModule


C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Using TensorFlow backend.


	YAML reader installed (version 0.17.17).
	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.3.0).


### Instantiation / Load

In [8]:
# Instantiate Fissa Module & Sub-Modules. 
# Sub-Module 1 is preparation, will contain raw data (PreparationModule)
# Sub-Module 2 is experiment, will contain separation data (SeparationModule)
# Sub-Module 3 is ProcessedTraces, just a container for processed signals
Data.Encoding.Fissa = FissaModule(data_folder=__data_folder, index_file = __index_file)
# This folder contains the suite2p/plane0/___.npy files as well as the saved registered files located in the suite2p/plane0/reg_tif folder
# Initialize
Data.Encoding.Fissa.pruneNonNeuronalROIs() # This step removes all non-neuronal data
Data.Encoding.Fissa.initializeFissa()
Data.Encoding.Fissa.loadFissaPrep()
Data.Encoding.Fissa.loadFissaSep()
Data.Encoding.Fissa.loadProcessedTraces()
# Cascade
Data.Encoding.Cascade = CascadeModule(Data.Encoding.Fissa.ProcessedTraces.detrended_merged_dFoF_result, Data.Encoding.Fissa.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")
Data.Encoding.Cascade.loadSpikeProb(load_path=Data.Encoding.Fissa.output_folder)
Data.Encoding.Cascade.loadSpikeInference(load_path=Data.Encoding.Fissa.output_folder)
Data.Encoding.Cascade.loadProcessedInferences(load_path=Data.Encoding.Fissa.output_folder)

Adjusting from 1-to-0 indexing
Initialized Fissa
Loading Fissa Preparation...
Finished Loading Fissa Prep
Loading Fissa Separation...
Finished Loading Fissa Sep
Loading Processed Traces...
Finished Loading Processed Traces.
Loading Spike Probabilities from Load Path...
Finished Loading Spike Probabilities
Loading Spike Times from Load Path...
Loading Discrete Approximations from Load Path...
Loading Processed Inferences...
Finished Loading Processed Traces.


In [ ]:
Data.saveHierarchy()

# Event-Based Analysis

### Import

In [9]:
from AnalysisModules.StaticProcessing import generateSpikeMatrix
from AnalysisModules.DecodingAnalysis import LogisticRegression
from AnalysisModules.BurrowFearConditioning import identifyTrialValence, reorganizeData

### Generate Spike Matrix

In [10]:
Data.Encoding.Cascade.ProcessedInferences.spike_matrix = generateSpikeMatrix(Data.Encoding.Cascade.spike_time_estimates, Data.Encoding.Cascade.spike_prob.shape[1])

Not Yet Implemented


Formatting Spike Matrix:   0%|          | 0/978 [00:00<?, ?it/s]

### Organize

In [12]:
# This just a dummy Log Reg to access function because im stupid rn
Data.Encoding.LogReg = LogisticRegression(NeuralData=Data.Encoding.Cascade.ProcessedInferences.spike_matrix, FeatureDataFile=__features_file)

#Organize spikes into trials x neurons x frames
Data.Encoding.organized_spikes, Data.Encoding.feature_index, Data.Encoding.organized_features = reorganizeData(Data.Encoding.Cascade.ProcessedInferences.spike_matrix, Data.Encoding.LogReg.feature_data, Data.Encoding.Cascade.frame_rate)

#Import valence data
__csIndexFile = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Behavior\\BehavioralExports\\DEM2_ENC_CS_INDEX.csv"
plus_trials, minus_trials = identifyTrialValence(__csIndexFile)
plus_trials = plus_trials[0]
minus_trials = minus_trials[0]

# Segregate organized spikes by valence
Data.Encoding.plus_spikes = Data.Encoding.organized_spikes[plus_trials, :, :]
Data.Encoding.minus_spikes = Data.Encoding.organized_spikes[minus_trials, :, :]
Data.Encoding.plus_features = Data.Encoding.organized_features[plus_trials, :, :]
Data.Encoding.minus_features = Data.Encoding.organized_features[minus_trials, :, :]

# Use 'Learned' Trials only
Data.Encoding.learned_plus_spikes = Data.Encoding.plus_spikes[5:11, :, :]
Data.Encoding.learned_minus_spikes = Data.Encoding.minus_spikes[5:11, :, :]
Data.Encoding.learned_plus_features = Data.Encoding.plus_features[5:11, :, :]
Data.Encoding.learned_minus_features = Data.Encoding.minus_features[5:11, :, :]

Instanced Logistic Regression


### Shuffle

In [13]:
_num_learned_trials = 5
_shuf_index = np.arange(5)
np.random.shuffle(_shuf_index)
Data.Encoding.shuffled_learned_plus_spikes = Data.Encoding.learned_plus_spikes[_shuf_index, :, :]
Data.Encoding.shuffled_learned_minus_spikes = Data.Encoding.learned_minus_spikes[_shuf_index, :, :]
Data.Encoding.shuffled_learned_plus_features = Data.Encoding.learned_plus_features[_shuf_index, :, :]
Data.Encoding.shuffled_learned_minus_features = Data.Encoding.learned_minus_features[_shuf_index, :, :]

### Use Trial Frames Only

In [14]:
Data.Encoding.new_plus = np.concatenate(Data.Encoding.shuffled_learned_plus_spikes[:, :, 492:886], axis=1)
Data.Encoding.new_minus = np.concatenate(Data.Encoding.shuffled_learned_minus_spikes[:, :, 492:886], axis=1)
Data.Encoding.new_plus_features = np.concatenate(Data.Encoding.shuffled_learned_plus_features[:, :, 492:886], axis=1)
Data.Encoding.new_minus_features = np.concatenate(Data.Encoding.shuffled_learned_minus_features[:, :, 492:886], axis=1)
Data.Encoding.new_neural_data = np.append(Data.Encoding.new_plus, Data.Encoding.new_minus, axis=1)
Data.Encoding.new_features_data = np.append(Data.Encoding.new_plus_features, Data.Encoding.new_minus_features, axis=1)

### Shuffle Frames

In [18]:
_shuf_frames = np.arange(Data.Encoding.new_neural_data.shape[1])
np.random.shuffle(_shuf_frames)
Data.Encoding.shuffled_new_neural_data = Data.Encoding.new_neural_data[:, _shuf_frames]
Data.Encoding.shuffled_new_features_data = Data.Encoding.new_features_data[:, _shuf_frames]

### Conduct Logistic Classification

In [16]:
Data.Encoding.LogReg = LogisticRegression(NeuralData=Data.Encoding.shuffled_new_neural_data, FeatureData=Data.Encoding.shuffled_new_features_data)
Data.Encoding.LogReg.label_data = Data.Encoding.LogReg.feature_data[1, :].copy()
Data.Encoding.LogReg.splitData()
Data.Encoding.LogReg.fitModel(penalty='l2', solver='lbfgs', max_iter=100000)
Data.Encoding.LogReg.assessFit()
Data.Encoding.LogReg.makeAllPredictions()
Data.Encoding.LogReg.commonAssessment()
Data.Encoding.LogReg.printAssessment()

Instanced Logistic Regression
Splitting data in training & testing sets
Data splits are: 80.0% training vs 20.0% testing
Fitting Logistic Regression...
Finished
Training Classification Accuracy is 0.9571700507614214
Printing Assessment of Model Performance:
('accuracy', 'training')  :  0.9571700507614214
('accuracy', 'testing')  :  0.9073604060913706
('precision', 'training')  :  0.9387351778656127
('precision', 'testing')  :  0.8487394957983193
('recall', 'training')  :  0.8203799654576857
('recall', 'testing')  :  0.6474358974358975
('f1', 'training')  :  0.8755760368663594
('f1', 'testing')  :  0.7345454545454546
('balanced_accuracy', 'training')  :  0.9041658863433006
('balanced_accuracy', 'testing')  :  0.8094774423888348
('AUC', 'training')  :  0.9041658863433008
('AUC', 'testing')  :  0.8094774423888348
('AUC_PR', 'training')  :  0.8031144566491738
('AUC_PR', 'testing')  :  0.6193013714661968


In [17]:
Data.Encoding.LogReg.plotROCs()